In [377]:
import os
import sys
import copy
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.special import logsumexp
from shapely.geometry import shape
from scipy.optimize import minimize

np.random.seed(888)

%matplotlib inline

In [378]:
# Get current working directory
cwd = os.getcwd()
# Remove all children directories
rd = os.path.join(cwd.split('stochastic-travel-demand-modelling/', 1)[0])
# Make sure directory ends with project's name
if not rd.endswith('stochastic-travel-demand-modelling'):
    rd = os.path.join(rd,'stochastic-travel-demand-modelling/')
    
sys.path.append('..')

from models.doubly_constrained.spatial_interaction_model import SpatialInteraction as DSIM

# Create data

## Origin and destination locations

In [382]:
# Set artificial info for destination sizes
dataset = 'synthetic'
var = 'dummy'
filepath = '../data/input/synthetic'
init_year = 0
final_year = 5
cambridge_data = True # False would generate Ward locations and randomly populate cost and flow matrix

# Take subset of origins and destinations 
N = 2
M = 3

In [388]:
if not cambridge_data:
    # Import processed ward coordinates
    ward_coordinates = gpd.read_file("../data/input/misc/ward_coordinates.json")
    n_locs = ward_coordinates.shape[0]
    
    if N == 1:
        origins = [ward_coordinates[['ward_new','geometry']][0].values]
    else:
        origins = ward_coordinates[['ward_new','geometry']][0:N].values
    if M == 1: 
        destinations = [ward_coordinates[['ward_new','geometry']][0].values]
    else:
        destinations = ward_coordinates[['ward_new','geometry']][0:M].values


    # Get origin and destination locations
    origin_locs = [[x[1].centroid.x,x[1].centroid.y] for x in origins]
    destination_locs = [[x[1].centroid.x,x[1].centroid.y] for x in destinations]

    # # Get origin and destination names
    origin_names = [x[0] for x in origins]
    destination_names = [x[0] for x in destinations]
    
else:
    # Manually create coordinates
    origins = [
        {
          "type": "Feature",
          "properties": {"name":"homerton"},
          "geometry": {
            "type": "Point",
            "coordinates": [
              0.15106201171874997,
              52.175510777101074
            ]
          }
        },
        {
          "type": "Feature",
          "properties": {"name":"hughes_hall"},
          "geometry": {
            "type": "Point",
            "coordinates": [
              0.13355255126953125,
              52.19950596108069
            ]
          }
        },
        {
          "type": "Feature",
          "properties": {"name":"newnham"},
          "geometry": {
            "type": "Point",
            "coordinates": [
              0.10282516479492186,
              52.196138997125985
            ]
          }
        }
      ]

    destinations = [
        {
          "type": "Feature",
          "properties": {"name":"west_cam"},
          "geometry": {
            "type": "Point",
            "coordinates": [
              0.08050918579101562,
              52.212760902425885
            ]
          }
        },
        {
          "type": "Feature",
          "properties": {"name":"central_cam"},
          "geometry": {
            "type": "Point",
            "coordinates": [
              0.12205123901367186,
              52.20424032262008
            ]
          }
        }
    ]

    # Get origin and destination locations
    origin_locs = [[shape(x['geometry']).x,shape(x['geometry']).y] for x in origins]
    destination_locs = [[shape(x['geometry']).x,shape(x['geometry']).y] for x in destinations]

    # Get origin and destination names
    origin_names = [x['properties']['name'] for x in origins]
    destination_names = [x['properties']['name'] for x in destinations]
    
    
    if N == 1:
        origins = [origins[0]]
    else:
        origins = origins[0:(N+1)]
    if M == 1: 
        destinations = [destinations[0]]
    else:
        destinations = destinations[0:(M+1)]

## Set true parameters

In [389]:
# Set true alpha and beta
true_alpha = 1.5#1.1
true_beta = 4
true_delta = 0
true_kappa = 1
true_gamma = 10000

## Create origin-destination data

In [403]:
# Set cost matrix
cost_matrix = np.ones((N,M))
if N != 2 and M != 2:
    cost_matrix = np.random.randint(1,(N+1)*(M+1), size=(N,M))
    #     for j in range(M):
#         cost_matrix[:,j] = np.ones(N)*np.random.randint(1,M+1, size=1)
else:
    cost_matrix[0,1] = 2
    cost_matrix[1,0] = 2

# Normalise cost matrix
c = cost_matrix/np.sum(cost_matrix)

# Set origin supply
origin_supply = np.random.randint(1,(N+1)*(M+1), size=(N)) #np.ones(N)
o = origin_supply/np.sum(origin_supply)

# Set destination demand
destination_demand = np.random.randint(1,(N+1)*(M+1), size=(M))
d = destination_demand/np.sum(destination_demand)

# Set destination sizes
initial_destination_sizes = (true_kappa + true_delta * M) * destination_demand
xd = np.log(initial_destination_sizes/np.sum(initial_destination_sizes))

# # Construct flow matrix
T = np.zeros((N,M))

# Define parameters
theta = np.array([true_alpha, true_beta, true_delta, true_gamma, true_kappa, 1])


In [404]:
# Construct flow matrix
si = DSIM(dataset)

value = si.infer_flows_ipf_procedure(T,
                                o,
                                d,
                                c,
                                np.exp(xd),
                                np.ones(N),
                                np.ones(M),
                                N,
                                M,
                                theta,
                                10000,
                                0.000000001,
                                False)

# for i in range(N):
#     for j in range(M):
#         _sum = 0
#         # Compute denominator
#         for jj in range(M):
#             _sum += np.exp(true_alpha*xd[j]-true_beta*c[i,jj])
#         # Compute estimated flow
#         T[i,j] = o[i]*np.exp(true_alpha*xd[j]-true_beta*c[i,j]) / _sum

<class 'numpy.ndarray'>
(2,)


In [405]:
print('Cost matrix')
print(pd.DataFrame(c))

print('Flow matrix')
print(pd.DataFrame(T))

Cost matrix
       0      1      2
0  0.125  0.250  0.125
1  0.250  0.125  0.125
Flow matrix
          0         1         2
0  0.069531  0.010380  0.045089
1  0.353545  0.143466  0.377988


In [399]:
print('estimated destination demand')
print(np.sum(T,axis=0))
print('actual destination demand')
print(d)
print('actual destination sizes')
print(np.exp(xd)/true_kappa)

estimated destination demand
[0.06666667 0.66666667 0.26666667]
actual destination demand
[0.06666667 0.66666667 0.26666667]
actual destination sizes
[0.06666667 0.66666667 0.26666667]


In [400]:
print('estimated origin supply')
print(np.sum(T,axis=1))
print('actual origin supply')
print(o)

estimated origin supply
[0.66666667 0.33333333]
actual origin supply
[0.66666667 0.33333333]


# Verify equilibria number and stability

This is based on 
- [1]: page 478 of Rijk, F. J. A., & Vorst, A. C. F. (1983). On the Uniqueness and Existence of Equilibrium Points in an Urban Retail Model. Environment and Planning A: Economy and Space, 15(4), 475–482. https://doi.org/10.1068/a150475
- [2]: page 391 of Rijk, F. J. A., & Vorst, A. C. F. (1983). Equilibrium points in an urban retail model and their connection with dynamical systems. Regional Science and Urban Economics, 13(3), 383–399. https://doi.org/10.1016/0166-0462(83)90024-8

In [401]:
if M == 2 and N == 2:
    cost = true_kappa*(c[0,0]**2 + c[0,1]**2)
    a = true_kappa*(c[0,0]*c[0,1])
    p = cost/(2*a)
    p0 = np.ones((M)) * N/(2*true_kappa)
    
    if c[0,1] != c[1,0]:
        raise ValueError(f"This is not a symmetric cost matrix {c}")
    
    if 0 < true_alpha and true_alpha <= 0.5:
        # Obtained from [1]
        print('There is exactly 1 unique possible solution to this dynamical system.')
    elif true_alpha < 1 or true_alpha >= (0.5*(1+p)):
        # Obtained from [1,2]
        print('There is exactly 1 possible solution to this dynamical system.')
    elif true_alpha > 1 and true_alpha < (0.5*(1+p)): 
        # Obtained from [2]
        print('There is exactly 3 possible solutions to this dynamical system.')
    else:
        print('This is an unknown case.')
        
    if true_alpha > (0.5*(1+p)):
        print(f'P0 {p0} is unstable.')
    elif true_alpha < (0.5*(1+p)): 
        print(f'P0 {p0} is stable.')
    else:
        print('This is an unknown case.')
        
    print(true_alpha,0.5*(1+p),p)
    
    print(np.min(np.exp(xd)))
    

# Export data to file

In [402]:
# Export cost matrix
np.savetxt(f'../data/input/{dataset}/cost_matrix.txt',cost_matrix)

# Export flow matrix
np.savetxt(f'../data/input/{dataset}/od_matrix.txt',T)

# Export origin-related data to txt
np.savetxt(f'../data/input/{dataset}/origin_supply.txt',origin_supply)
np.savetxt(f'../data/input/{dataset}/origin_locations.txt',origin_locs)
np.savetxt(f'../data/input/{dataset}/origins.txt',origin_names,fmt="%s")

# Export destination-related data to txt
np.savetxt(f'../data/input/{dataset}/destination_demand.txt',destination_demand)
np.savetxt(f'../data/input/{dataset}/destination_locations.txt',destination_locs)
np.savetxt(f'../data/input/{dataset}/destinations.txt',destination_names,fmt="%s")

# Export destination sizes to txt
np.savetxt(f'../data/input/{dataset}/initial_destination_sizes.txt',initial_destination_sizes)
# np.savetxt(f'../data/input/{dataset}/final_destination_sizes.txt',final_destination_sizes)

# Export semantic meaning of destination sizes
with open(f"../data/input/{dataset}/destination_sizes_info.txt", "w") as text_file:
    print(f"destination_sizes_data_filepath,{filepath}", file=text_file)
    print(f"destination_sizes_variable,{var}", file=text_file)
    print(f"initial_destination_sizes_year,{str(init_year)}", file=text_file)
#     print(f"final_destination_sizes_year,{str(final_year)}", file=text_file)
    print(f"true_alpha,{str(true_alpha)}", file=text_file)
    print(f"true_beta,{str(true_beta)}", file=text_file)
    print(f"true_delta,{str(true_delta)}", file=text_file)
    print(f"true_kappa,{str(true_kappa)}", file=text_file)